## Quickstart: Better together Snowflake Semantic view and Amazon Quick sight 

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


#### Create Snowflake environment (warehouse, database, view and roles)

In [ ]:
-- =============================================
-- PART 1: Create the account level objects for semantic view quick start
-- =============================================
USE ROLE ACCOUNTADMIN;
-- Create role for semantic view quick start
CREATE ROLE IF NOT EXISTS quickstart_role COMMENT = 'Role for semantic view quick start demo';

-- Set variables for user
SET my_user = CURRENT_USER();

--Grant role to your user
GRANT ROLE quickstart_role TO USER IDENTIFIER($my_user);
GRANT CREATE INTEGRATION ON ACCOUNT TO ROLE quickstart_role;
GRANT EXECUTE TASK ON ACCOUNT TO ROLE quickstart_role;
GRANT EXECUTE MANAGED TASK ON ACCOUNT TO ROLE quickstart_role;
GRANT MONITOR EXECUTION ON ACCOUNT TO ROLE quickstart_role;

-- Create warehouse, database, schema and grant role
CREATE WAREHOUSE IF NOT EXISTS workshopwh WITH WAREHOUSE_SIZE = 'XSMALL' AUTO_SUSPEND = 60 AUTO_RESUME = TRUE COMMENT = 'Warehouse for semantic view quick start demo';
CREATE DATABASE IF NOT EXISTS movies;
GRANT OWNERSHIP ON DATABASE movies TO ROLE quickstart_role COPY CURRENT GRANTS;
GRANT OWNERSHIP ON SCHEMA movies.PUBLIC TO ROLE quickstart_role COPY CURRENT GRANTS;
GRANT OWNERSHIP ON WAREHOUSE workshopwh TO ROLE quickstart_role COPY CURRENT GRANTS;

-- Grant privileges to create semantic views
GRANT CREATE SEMANTIC VIEW ON SCHEMA movies.PUBLIC TO ROLE quickstart_role;
GRANT CREATE STAGE ON SCHEMA movies.PUBLIC TO ROLE quickstart_role;

In [ ]:
-- Verify the below information
SELECT
  CURRENT_DATABASE() AS current_db,
  CURRENT_SCHEMA()   AS current_schema,
  CURRENT_ROLE()     AS current_role,
  CURRENT_USER()     AS current_user;

In [ ]:
-- Set variables for the specified role, database, and schema
SET my_role = 'quickstart_role';
SET my_db = 'movies';
SET my_schema = 'movies.PUBLIC';
SET my_full_schema = $my_db || '.' || $my_schema;

In [ ]:
-- ==============================================
-- PART 2: DATA SETUP (as quickstart_role)
-- ==============================================
-- create demo table for our movie data, we will surface this in dashboard
CREATE TABLE if not exists movies_dashboard (
    movie_id NUMBER,
    movie_title VARCHAR,
    movie_release_year INTEGER,
    genre VARCHAR,
    user_rating FLOAT,
    rating_timestamp TIMESTAMP_NTZ,
    user_id NUMBER,
    user_firstname VARCHAR,
    user_lastname VARCHAR,
    user_city VARCHAR,
    user_state VARCHAR,
    user_country VARCHAR,
    user_email VARCHAR,
    user_phonenumber VARCHAR,
    interaction_timestamp NUMBER,
    interaction_type VARCHAR
);

-- create stage for data loading 
CREATE OR REPLACE STAGE MOVIEDASHBOARD URL = 's3://hol-qs-bucket/' FILE_FORMAT = (TYPE = 'csv');
COPY INTO movies_dashboard
FROM
    @MOVIEDASHBOARD/movies_dashboard.csv FILE_FORMAT =(
        TYPE = 'csv' FIELD_DELIMITER = ',' SKIP_HEADER = 1
    );
-- Create stage for docs
CREATE OR REPLACE STAGE DOCS DIRECTORY = (ENABLE = true) ENCRYPTION = (TYPE = 'SNOWFLAKE_SSE');
-- Create stage for semantic models
CREATE STAGE IF NOT EXISTS MODELS DIRECTORY = (ENABLE = TRUE) COMMENT = 'Stage for semantic model files';

-- let's verify the data load
select distinct movie_title from movies_dashboard;


-- verify 773 records been loaded
select count(*) from movies_dashboard;

In [ ]:
-- verify 773 records been loaded
select count(*) from movies_dashboard;

In [ ]:
-- let's verify the data load
select distinct movie_title from movies_dashboard;

### Now we will create the semantic view for the movie dashboard

From Snowsight
* Select **AI & ML** 
* Select **Cortex Analyst**
* Click on the `Create new` drop down 
* Ensure to select the  `QUICKSTART_ROLE` and `WORKSHOPSH` created earlier
* In the **Location to  store**: Select `Movies` database and `Public` Schema
* Enter `MOVIES_SV` as the name of the semantic view 
* Click **NEXT** to continue


Confirm the creation of the semantic view before proceeding, under: Select database schema 
Database: MOVIES
Schema: PUBLIC
Semantic Views: MOVIES_SV


In [ ]:
-- ==============================================
-- PART 3: Proceed after creation of Semantic View
-- ==============================================
-- Sample Test Queries for MOVIES_SV Semantic View
-- 1. Movies by genre with average ratings

SELECT
    *
FROM
    SEMANTIC_VIEW (
        MOVIES_SV DIMENSIONS MOVIES_DASHBOARD.movie_title METRICS AVG(MOVIES_DASHBOARD.user_rating) AS avg_rating
    )
ORDER BY
    avg_rating DESC
LIMIT
    10;

In [ ]:
-- 2. Top rated movies
SELECT * FROM SEMANTIC_VIEW (
  MOVIES_SV
  DIMENSIONS MOVIES_DASHBOARD.movie_title, MOVIES_DASHBOARD.movie_release_year
  METRICS AVG(MOVIES_DASHBOARD.user_rating) AS avg_rating, COUNT(MOVIES_DASHBOARD.user_rating) AS rating_count
)
WHERE rating_count > 10
ORDER BY avg_rating DESC
LIMIT 10;

In [ ]:
import boto3
import json
import os

# Load AWS profile
AWS_PROFILE = os.getenv('AWS_PROFILE', 'quick_suite_sa')
session = boto3.Session(profile_name=AWS_PROFILE)

# Initialize clients
quicksight = session.client('quicksight', region_name='us-east-1')
sts = session.client('sts')
account_id = sts.get_caller_identity()['Account']

print(f"Using AWS Account ID: {account_id}")

# Delete existing data source if it exists
datasource_id = 'movies-snowflake-datasource'
try:
    quicksight.delete_data_source(
        AwsAccountId=account_id,
        DataSourceId=datasource_id
    )
    print(f"Existing data source deleted: {datasource_id}")
except quicksight.exceptions.ResourceNotFoundException:
    print(f"No existing data source found: {datasource_id}")

# Snowflake configuration (from secrets)
SNOWFLAKE_CONFIG = {
    'account': 'awspartner',
    'database': 'MOVIES',
    'warehouse': 'WORKSHOPWH',
    'user': 'QUICKSA',
    'password': 'YourSnowflakePassword'  # Replace with your actual password or use a secure method to retrieve it
}

# Create Snowflake Data Source
datasource_id = 'movies-snowflake-datasource'
datasource_response = quicksight.create_data_source(
    AwsAccountId=account_id,
    DataSourceId=datasource_id,
    Name='Movies Snowflake Data Source',
    Type='SNOWFLAKE',
    DataSourceParameters={
        'SnowflakeParameters': {
            'Host': f"{SNOWFLAKE_CONFIG['account']}.snowflakecomputing.com",
            'Database': SNOWFLAKE_CONFIG['database'],
            'Warehouse': SNOWFLAKE_CONFIG['warehouse']
        }
    },
    Credentials={
        'CredentialPair': {
            'Username': SNOWFLAKE_CONFIG['user'],
            'Password': SNOWFLAKE_CONFIG['password']
        }
    }
)

print(f"Data Source Created: {datasource_response['DataSourceId']}")

# Delete existing dataset if it exists
import time
dataset_id = 'movies-dashboard-dataset'
try:
    quicksight.delete_data_set(
        AwsAccountId=account_id,
        DataSetId=dataset_id
    )
    print(f"Existing dataset deleted: {dataset_id}, waiting for deletion to complete...")
    time.sleep(10)  # Wait for deletion to complete
except quicksight.exceptions.ResourceNotFoundException:
    print(f"No existing dataset found: {dataset_id}")

# Create Dataset with base table
dataset_response = quicksight.create_data_set(
    AwsAccountId=account_id,
    DataSetId=dataset_id,
    Name='Movies Dashboard Dataset',
    PhysicalTableMap={
        'moviesdashboard': {
            'RelationalTable': {
                'DataSourceArn': f"arn:aws:quicksight:us-east-1:{account_id}:datasource/{datasource_id}",
                'Catalog': SNOWFLAKE_CONFIG['database'],
                'Schema': 'PUBLIC',
                'Name': 'MOVIES_DASHBOARD',
                'InputColumns': [
                    {'Name': 'GENRE', 'Type': 'STRING'},
                    {'Name': 'INTERACTION_TIMESTAMP', 'Type': 'INTEGER'},
                    {'Name': 'INTERACTION_TYPE', 'Type': 'STRING'},
                    {'Name': 'MOVIE_ID', 'Type': 'INTEGER'},
                    {'Name': 'MOVIE_RELEASE_YEAR', 'Type': 'INTEGER'},
                    {'Name': 'MOVIE_TITLE', 'Type': 'STRING'},
                    {'Name': 'USER_CITY', 'Type': 'STRING'},
                    {'Name': 'USER_COUNTRY', 'Type': 'STRING'},
                    {'Name': 'USER_EMAIL', 'Type': 'STRING'},
                    {'Name': 'USER_FIRSTNAME', 'Type': 'STRING'},
                    {'Name': 'USER_ID', 'Type': 'INTEGER'},
                    {'Name': 'USER_LASTNAME', 'Type': 'STRING'},
                    {'Name': 'USER_PHONENUMBER', 'Type': 'STRING'},
                    {'Name': 'USER_STATE', 'Type': 'STRING'},
                    {'Name': 'RATING_TIMESTAMP', 'Type': 'DATETIME'},
                    {'Name': 'USER_RATING', 'Type': 'DECIMAL'}
                ]
            }
        }
    },
    LogicalTableMap={
        'movieslogical': {
            'Alias': 'Movies Data',
            'Source': {
                'PhysicalTableId': 'moviesdashboard'
            },
            'DataTransforms': [
                {
                    'ProjectOperation': {
                        'ProjectedColumns': [
                            'GENRE',
                            'INTERACTION_TIMESTAMP',
                            'INTERACTION_TYPE',
                            'MOVIE_ID',
                            'MOVIE_RELEASE_YEAR',
                            'MOVIE_TITLE',
                            'USER_CITY',
                            'USER_COUNTRY',
                            'USER_EMAIL',
                            'USER_FIRSTNAME',
                            'USER_ID',
                            'USER_LASTNAME',
                            'USER_PHONENUMBER',
                            'USER_STATE',
                            'RATING_TIMESTAMP',
                            'USER_RATING'
                        ]
                    }
                }
            ]
        }
    },
    ImportMode='SPICE',
    FieldFolders={
        'Movie Information': {
            'description': 'Fields related to movie details',
            'columns': ['MOVIE_ID', 'MOVIE_TITLE', 'MOVIE_RELEASE_YEAR', 'GENRE']
        },
        'User Information': {
            'description': 'Fields related to user details',
            'columns': ['USER_ID', 'USER_FIRSTNAME', 'USER_LASTNAME', 'USER_EMAIL', 
                       'USER_PHONENUMBER', 'USER_CITY', 'USER_STATE', 'USER_COUNTRY']
        },
        'Interactions': {
            'description': 'Fields related to user interactions',
            'columns': ['INTERACTION_TYPE', 'INTERACTION_TIMESTAMP', 'RATING_TIMESTAMP', 'USER_RATING']
        }
    },
    DataSetUsageConfiguration={
        'DisableUseAsDirectQuerySource': True,
        'DisableUseAsImportedSource': False
    }
)

print(f"Dataset Created: {dataset_response['DataSetId']}")

# Create SPICE ingestion
import time
ingestion_id = f"ingestion-{int(time.time())}"
ingestion_response = quicksight.create_ingestion(
    AwsAccountId=account_id,
    DataSetId=dataset_id,
    IngestionId=ingestion_id,
    IngestionType='FULL_REFRESH'
)

print(f"SPICE Ingestion Started: {ingestion_response['IngestionId']}")
print(f"Ingestion Status: {ingestion_response['IngestionStatus']}")



Using AWS Account ID: 905178920046
Existing data source deleted: movies-snowflake-datasource
Data Source Created: movies-snowflake-datasource
Existing dataset deleted: movies-dashboard-dataset, waiting for deletion to complete...
Dataset Created: movies-dashboard-dataset
SPICE Ingestion Started: ingestion-1768424390
Ingestion Status: QUEUED


In [3]:
# Share dataset with specific user
share_user = 'ElevatedAccess/wangzyn-Isengard' #os.getenv('QUICKSIGHT_USER', 'Admin/quicksight-sa')  # Format: namespace/username
permissions_response = quicksight.update_data_set_permissions(
    AwsAccountId=account_id,
    DataSetId=dataset_id,
    GrantPermissions=[
        {
            'Principal': f"arn:aws:quicksight:us-east-1:{account_id}:user/default/{share_user}",
            'Actions': [
                'quicksight:UpdateDataSetPermissions',
                'quicksight:DescribeDataSet',
                'quicksight:DescribeDataSetPermissions',
                'quicksight:PassDataSet',
                'quicksight:DescribeIngestion',
                'quicksight:ListIngestions',
                'quicksight:UpdateDataSet',
                'quicksight:DeleteDataSet',
                'quicksight:CreateIngestion',
                'quicksight:CancelIngestion'
            ]
        }
    ]
)

print(f"Dataset shared with user: {share_user}")
print("QuickSight Data Source and Dataset created successfully!")
print(f"Data Source ARN: arn:aws:quicksight:us-east-1:{account_id}:datasource/{datasource_id}")
print(f"Dataset ARN: arn:aws:quicksight:us-east-1:{account_id}:dataset/{dataset_id}")
print(f"Ingestion ARN: arn:aws:quicksight:us-east-1:{account_id}:dataset/{dataset_id}/ingestion/{ingestion_id}")


Dataset shared with user: ElevatedAccess/wangzyn-Isengard
QuickSight Data Source and Dataset created successfully!
Data Source ARN: arn:aws:quicksight:us-east-1:905178920046:datasource/movies-snowflake-datasource
Dataset ARN: arn:aws:quicksight:us-east-1:905178920046:dataset/movies-dashboard-dataset
Ingestion ARN: arn:aws:quicksight:us-east-1:905178920046:dataset/movies-dashboard-dataset/ingestion/ingestion-1768424390


# Field Descriptions

## Movie Fields

- **GENRE**: The type of movie genre, such as action, comedy, drama, etc., that categorizes the movie's style, tone, and content.
- **MOVIE_ID**: Unique identifier for each movie in the database.
- **MOVIE_RELEASE_YEAR**: The year in which the movie was released.
- **MOVIE_TITLE**: The title of the movie.

## User Fields

- **USER_ID**: Unique identifier for the user who interacted with the movie.
- **USER_FIRSTNAME**: The first name of the user who interacted with the movie.
- **USER_LASTNAME**: The last name of the user who interacted with the movie.
- **USER_EMAIL**: The email address of the user who interacted with the movie content.
- **USER_PHONENUMBER**: The phone number of the user who interacted with the movie content.
- **USER_CITY**: The city where the user is located.
- **USER_STATE**: The state in which the user is located.
- **USER_COUNTRY**: The country where the user is located.

## Interaction Fields

- **INTERACTION_TIMESTAMP**: The timestamp when a user interacted with the movie, representing the number of seconds that have elapsed since January 1, 1970, at 00:00:00 UTC.
- **INTERACTION_TYPE**: The type of interaction a user had with a movie, either clicking on it or watching it.

## Rating Fields

- **RATING_TIMESTAMP**: The date and time when a movie rating was recorded.
- **USER_RATING**: Average rating given by users to a movie, on a scale of 1 to 5.

---

**Note**: QuickSight API doesn't directly support setting field descriptions via `create_data_set`. Field descriptions are typically set through the QuickSight UI or by using calculated fields. This structure provides the foundation for the dataset.

![edit field descriptions](images/edit_desc.png)
![edit field descriptions 2](images/edit_desc_2.png)


1. Go to the Amazon Quick Suite on the console .
2. On the top right hand corner, click on the user icon → US West, and ensure that US West (Oregon) is selected.
![open quick suite](https://static.us-east-1.prod.workshops.aws/public/7eed92ee-f0ee-4194-b29e-1764804d6271/static/4-2-1-region.png) 
3. On the left panel, click on Datasets → Data Sources → Create Data Source, then from the options scroll and select Snowflake.
![create data source](https://static.us-east-1.prod.workshops.aws/public/7eed92ee-f0ee-4194-b29e-1764804d6271/static/4-2-2-data-sources.png)
![create data source2](https://static.us-east-1.prod.workshops.aws/public/7eed92ee-f0ee-4194-b29e-1764804d6271/static/4-2-3-snowflake-data-source.png)
4. Enter the following details:
   - Data source name: **movies-snowflake-datasource**
   - Snowflake account: *<your_snowflake_account>*
   - Warehouse: **WORKSHOPWH**
   - Database: **MOVIES**
   - Schema: **PUBLIC**
   - Username: *<your_snowflake_username>*
   - Password: *<your_snowflake_password>*
5. Click on **Create data source**.
6. Next, create a dataset from the data source you just created. On the left panel
    - Click on Datasets → New Dataset → Snowflake-QuickStart.
    - Select **Use custom SQL**.
    - In the dialog box, enter the following SQL query to select data from the semantic view you created earlier:
      ```sql
      SELECT * FROM "MOVIES"."PUBLIC"."movies_dashboard "
      ```
    - Click on **Confirm query**.
    - On the next screen, click on **Visualize** to create an analysis using this dataset.

In Quick Suite UI, add a table visual, and then add the dimensions and fact to build the dashboard
![build analysis](images/build_analysis.png) 